In [1]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("data/rel18").load_data()

In [2]:
docs_str = []
for doc in documents:
  docs_str.append(doc.text)

### Indexing

In [3]:
len(docs_str)

554

In [4]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
RAG.index(
    collection=docs_str,
    index_name="ITU RAG 150",
    max_document_length=150,
    split_documents=True,
)



[Jul 23, 09:05:35] #> Note: Output directory .ragatouille/colbert/indexes/ITU RAG 150 already exists


[Jul 23, 09:05:35] #> Will delete 78 files already at .ragatouille/colbert/indexes/ITU RAG 150 in 20 seconds...
[Jul 23, 09:05:57] [0] 		 #> Encoding 115738 passages..
[Jul 23, 09:07:59] [0] 		 avg_doclen_est = 99.74101257324219 	 len(local_sample) = 115,738
[Jul 23, 09:08:03] [0] 		 Creating 65,536 partitions.
[Jul 23, 09:08:03] [0] 		 *Estimated* 43,491,370 embeddings.
[Jul 23, 09:08:03] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/ITU RAG 150/plan.json ..
Clustering 11493825 points in 128D to 65536 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.91 s
  Iteration 3 (74.22 s, search 72.09 s): objective=1.78645e+06 imbalance=1.342 nsplit=1        
[Jul 23, 09:09:25] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/ubuntu/miniconda3/envs/itu_qna/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Jul 23, 09:09:25] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/ubuntu/miniconda3/envs/itu_qna/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[0.027, 0.028, 0.028, 0.025, 0.025, 0.027, 0.028, 0.026, 0.027, 0.027, 0.025, 0.027, 0.026, 0.026, 0.026, 0.029, 0.025, 0.025, 0.025, 0.026, 0.027, 0.027, 0.026, 0.027, 0.026, 0.026, 0.028, 0.028, 0.027, 0.029, 0.027, 0.028, 0.027, 0.025, 0.027, 0.024, 0.028, 0.026, 0.026, 0.031, 0.028, 0.027, 0.028, 0.027, 0.026, 0.026, 0.025, 0.029, 0.029, 0.026, 0.027, 0.026, 0.027, 0.028, 0.026, 0.027, 0.029, 0.027, 0.03, 0.026, 0.026, 0.027, 0.027, 0.028, 0.028, 0.028, 0.027, 0.028, 0.025, 0.027, 0.029, 0.025, 0.025, 0.029, 0.026, 0.027, 0.028, 0.027, 0.027, 0.028, 0.028, 0.027, 0.026, 0.027, 0.026, 0.027, 0.027, 0.026, 0.025, 0.03, 0.026, 0.028, 0.025, 0.028, 0.027, 0.027, 0.03, 0.025, 0.028, 0.026, 0.026, 0.028, 0.027, 0.027, 0.028, 0.026, 0.027, 0.026, 0.027, 0.025, 0.029, 0.029, 0.027, 0.026, 0.026, 0.026, 0.028, 0.028, 0.026, 0.028, 0.025, 0.026, 0.027, 0.028, 0.025, 0.028, 0.026, 0.025]


...

18it [08:02, 26.78s/it]
100%|██████████| 18/18 [00:00<00:00, 356.33it/s]


[Jul 23, 09:17:29] #> Optimizing IVF to store map from centroids to list of pids..
[Jul 23, 09:17:29] #> Building the emb2pid mapping..
[Jul 23, 09:17:30] len(emb2pid) = 43498000


100%|██████████| 65536/65536 [00:01<00:00, 37047.88it/s]


[Jul 23, 09:17:33] #> Saved optimized IVF to .ragatouille/colbert/indexes/ITU RAG 150/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/ITU RAG 150'

In [5]:
results = RAG.search(query="What does the UE provide to the AS for slice aware cell reselection?", k=7)
results

Loading searcher for index ITU RAG 150 for the first time... This may take a few seconds
[Jul 23, 09:17:36] #> Loading codec...
[Jul 23, 09:17:36] #> Loading IVF...
[Jul 23, 09:17:37] #> Loading doclens...


100%|██████████| 18/18 [00:00<00:00, 1692.46it/s]

[Jul 23, 09:17:37] #> Loading codes and residuals...



100%|██████████| 18/18 [00:01<00:00, 15.74it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What does the UE provide to the AS for slice aware cell reselection?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2515,  1996,  1057,  2063,  3073,  2000,  1996,
         2004,  2005, 14704,  5204,  3526, 24501, 12260,  7542,  1029,   102,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



[{'content': 'The NSAG specific cell reselection information is provided per frequency per NSAG. If NSAG specific cell reselection information is provided in dedicated signalling, the UE shall ignore NSAG specific cell reselection information provided in system information.\n\nIn the UE, NAS provides the NSAG information and their priorities to be considered during cell reselection to the AS. When a UE supports slice aware cell reselection, and NSAG specific cell reselection information is provided to the UE, then the UE performs the slice aware cell reselection. The details of slice aware cell reselection are specified in TS 38.304.',
  'score': 26.015625,
  'rank': 1,
  'document_id': '6e10f59e-4731-4fd1-958c-01dc075082ae',
  'passage_id': 435261},
 {'content': 'The details of slice aware cell reselection are specified in TS 38.304. In general, the UE can derive reselection priorities for slice aware cell reselection, and then perform cell reselection evaluation using legacy evaluati

In [27]:
# set random seeds for reproducibility
import random
import numpy as np
import torch
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [7]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained("alexgichamba/phi-2-finetuned-qa-lora-r32-a16_longcontext")
base_model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2").to('cuda')
model = PeftModel.from_pretrained(base_model, "alexgichamba/phi-2-finetuned-qa-lora-r32-a16_longcontext").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
import json
# Read questions from the JSON file
with open('data/TeleQnA_testing1.txt', 'r') as file1:
  with open('data/questions_new.txt', 'r') as file2:
    questions = json.load(file1)
    questions.update(json.load(file2))

In [29]:
len(questions)

2366

In [30]:
first_key = next(iter(questions))
first_value = questions[first_key]
first_value

{'question': 'When can a gNB transmit a DL transmission(s) on a channel after initiating a channel occupancy? [3GPP Release 17]',
 'option 1': 'Regardless of the duration of the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB.',
 'option 2': 'If the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB is more than a threshold.',
 'option 3': 'Both option 1 and option 2',
 'option 4': 'None of the above',
 'category': 'Standards specifications'}

In [31]:
options = [(k, v) for k, v in first_value.items() if k.startswith("option") and v is not None]
options

[('option 1',
  'Regardless of the duration of the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB.'),
 ('option 2',
  'If the gap between the DL transmission(s) and any previous transmission(s) corresponding to the channel occupancy initiated by the gNB is more than a threshold.'),
 ('option 3', 'Both option 1 and option 2'),
 ('option 4', 'None of the above')]

In [32]:
res = RAG.search(query=first_value['question'], k=7)
len(res)

7

In [33]:
type(first_value)

dict

In [34]:
def create_prompt(question, options, context, abbreviations):
    options_text = "\n".join([f"Option {i+1}: {opt[1]}" for i, opt in enumerate(options)])
    # abbreviations is a list of dictionaries of form {"abbreviation": "full form"}
    abbreviations_text = "\n".join([f"{list(abbrev.keys())[0]}: {list(abbrev.values())[0]}" for abbrev in abbreviations])
    prompt = (
        f"Instruct: You will answer each question correctly using the context below\n"
        # f"The output should be in the format: Option <Option id>\n"
        # f"Provide the answer to the following multiple choice question in the specified format.\n\n"
        f"Context: {context}\n\n"
        f"Abbreviations:\n{abbreviations_text}\n\n"
        f"Question: {question}\n"
        f"{options_text}\n"
        f"Answer: Option"
    )
    return prompt

In [35]:
def generate_answer(question, options, context, abbreviations, model, tokenizer):
    prompt = create_prompt(question, options, context, abbreviations)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

    # Ensure the pad token is set
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    attention_mask = input_ids.ne(tokenizer.pad_token_id).long().to('cuda')  # Set attention mask

    # Generate the answer with appropriate parameters
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=10,  # Limit the number of new tokens generated
        pad_token_id=tokenizer.eos_token_id,  # Handle padding correctly
        num_beams=1,  # Use beam search to improve quality of generated answers
        early_stopping=True  # Stop early when enough beams have reached EOS
    )
    answer = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True)
    
    # if number of input tokens exceed 2048
    # if input_ids.shape[1] > 2048:
    #     print(f"WARNING: Input length {input_ids.shape[1]} exceeds 2048 tokens")
    return answer


In [36]:
from data.prepare_docs import find_appearing_abbreviations
print(find_appearing_abbreviations(first_value))
type(find_appearing_abbreviations(first_value))

[{'DL': 'Down Link'}, {'gNB': 'next Generation Node B'}]


list

In [37]:
import re
q = re.sub(r'\s*\[.*?\]\s*$', '', first_value['question'])
# q = first_value['question']
ans = generate_answer(q, options, " ".join([result['content'] for result in results]), find_appearing_abbreviations(first_value), model, tokenizer)
print(ans)

option 2: If the gap between the DL transmission


In [38]:
import re
# First search for the full pattern
def parse_answer(response):
  match = re.search(r'Answer:\s*Option\s*(\d+)', response, re.IGNORECASE)
  if match:
      answer = f"Option {match.group(1)}"
  else:
      # Try another pattern if the first one fails
      match = re.search(r'(\d+)', response, re.IGNORECASE)
      if match:
          answer = f"Option {match.group(1)}"
      else:
          answer = "Error"
  return answer


In [39]:
import csv
from tqdm import tqdm

responses = []

# Loop through each question and get the response
for q_id, q_data in tqdm(questions.items(), desc="Processing questions"):
    q_id_number = q_id.split()[1]
    question_text = q_data["question"]
    question_text = re.sub(r'\s*\[.*?\]\s*$', '', question_text)
    # options = [v for k, v in q_data.items() if k.startswith("option")]
    options = [(k, v) for k, v in q_data.items() if k.startswith("option") and v is not None]

    # Retrieve context using ColBERT search
    results = RAG.search(query=question_text, k=13)
    context = " ".join([result['content'] for result in results])

    abbreviations = find_appearing_abbreviations(q_data)
    # Generate the answer using the loaded model
    response = generate_answer(question_text, options, context, abbreviations, model, tokenizer)
    answer = parse_answer(response)

    # Extract the answer ID from the response
    match = re.search(r'Option (\d+)', answer)
    if match:
        try:
            answer_id = int(match.group(1))
            print(f"Answer ID: {answer_id}")
            responses.append([q_id_number, answer_id, "Phi-2"])
        except (KeyError, IndexError, ValueError) as e:
            responses.append([q_id_number, "Error", "Phi-2"])
            print(f"Error processing question {q_id}: {answer}")
    else:
        responses.append([q_id_number, "Error", "Phi-2"])
        print(f"Error processing question {q_id_number}: {answer}")

# Save responses to a CSV file
with open('output_results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Question_ID", "Answer_ID", "Task"])
    csvwriter.writerows(responses)

print("Processing complete. Responses saved to 'output_results.csv'.")


Processing questions:   0%|          | 1/2366 [00:01<53:11,  1.35s/it]

Answer ID: 3


Processing questions:   0%|          | 2/2366 [00:02<54:36,  1.39s/it]

Answer ID: 5


Processing questions:   0%|          | 3/2366 [00:04<53:28,  1.36s/it]

...

Answer ID: 2


Processing questions: 100%|█████████▉| 2365/2366 [54:45<00:01,  1.35s/it]

Answer ID: 5


Processing questions: 100%|██████████| 2366/2366 [54:47<00:00,  1.39s/it]

Answer ID: 5
Processing complete. Responses saved to 'output_results.csv'.


grade the 366 qs

In [41]:
import csv
# Load the questions dataset from the JSON file
with open('data/366qs.txt', 'r') as rubric:
    qs_w_ans = json.load(rubric)

# Load the responses from the CSV file
responses = []
with open('output_results.csv', 'r') as answers:
    reader = csv.DictReader(answers)
    for row in reader:
        responses.append(row)

# Initialize score
correct_answers = 0
total_questions = len(responses[:366])

# track question_ids for failed questions
failed_questions = []
# Compare the responses with the correct answers for first 366 questions
for response in responses[:366]:
    question_id = response['Question_ID']
    answer_id = response['Answer_ID']
    task = response['Task']
    
    # Find the corresponding question in the JSON data
    question_key = f"question {question_id}"
    if question_key in qs_w_ans:
        correct_answer = qs_w_ans[question_key]['answer']
        # Extract the correct option number from the correct answer string
        correct_option_number = correct_answer.split()[1].replace(":", "")
        
        # Check if the given answer matches the correct answer
        if answer_id == correct_option_number:
            correct_answers += 1
        else:
            # append questionid and answerid to failed questions
            failed_questions.append((question_id, answer_id))

# Calculate the score
score = (correct_answers / total_questions) * 100

# Print the results
print(f"Total Questions: {total_questions}")
print(f"Correct Answers: {correct_answers}")
print(f"Score: {score:.2f}%")
# write failed questions to a file
with open('failed_questions.txt', 'w') as file:
    for question_id, answer_id in failed_questions:
        file.write(f"{question_id} {answer_id}\n")

Total Questions: 366
Correct Answers: 297
Score: 81.15%


In [ ]:
# def append_dummy_data(csv_filename):
#     dummy_task = "Phi-2"
#     dummy_entries = [(dummy_id, 0, dummy_task) for dummy_id in range(10000, 12000)]

#     try:
#         # Open the existing CSV file and append dummy data
#         with open(csv_filename, "a", newline='') as csvfile:
#             csv_writer = csv.writer(csvfile)
#             for entry in dummy_entries:
#                 csv_writer.writerow(entry)
#         print("Dummy data has been appended to the CSV file.")
#     except Exception as e:
#         print("Encountered an error while appending dummy data.")
#         print(e)

# # Assuming 'output_results.csv' is the CSV file to which we need to append dummy data
# csv_filename = "output_results.csv"
# append_dummy_data(csv_filename)

## Generate static context for training set

In [ ]:
# from tqdm import tqdm
# import re

# files = [("data/qs_train.txt", "data/qs_train_with_context.txt"), ("data/366qs.txt", "data/366qs_with_context.txt")]

# for training_set_file, output_file in files:
#     # Load the questions dataset from the JSON file
#     with open(training_set_file, 'r') as file:
#         questions = json.load(file)

#     for q_id, q_data in tqdm(questions.items(), desc="Processing questions"):
#         q_id_number = q_id.split()[1]
#         question_text = q_data["question"]
#         question_text = re.sub(r'\s*\[.*?\]\s*$', '', question_text)
#         results = RAG.search(query=question_text, k=7)
#         context = " ".join([result['content'] for result in results])
#         q_data["context"] = context

#     with open(output_file, 'w') as file:
#         json.dump(questions, file, indent=4)


In [ ]:
# results = RAG.search(query="What does the NEF notify to the AF after determining the suitable DNAI(s)?", k=3)
# results_exp = " ".join([result['content'] for result in results])
# print(len(results_exp))

## Hybrid: ColBERT+BM25

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
# initialize node parser
splitter = SentenceSplitter(chunk_size=150, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=1,)

In [ ]:
# bm25_retriever.persist("./bm25_retriever_150")
# loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever_150")

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retrieved_nodes = bm25_retriever.retrieve(
    "What does the UE provide to the AS for slice aware cell reselection?"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=2000)

In [ ]:
def combine_contexts(rag_context, bm25_context):
    rag = " ".join([result['content'] for result in rag_context])
    bm25 = " ".join([node.text for node in bm25_context])
    return bm25 + "\n" + rag

In [ ]:
import csv
from tqdm import tqdm

responses = []

# Loop through each question and get the response
for q_id, q_data in tqdm(questions.items(), desc="Processing questions"):
    q_id_number = q_id.split()[1]
    question_text = q_data["question"]
    question_text = re.sub(r'\s*\[.*?\]\s*$', '', question_text)
    # options = [v for k, v in q_data.items() if k.startswith("option")]
    options = [(k, v) for k, v in q_data.items() if k.startswith("option") and v is not None]

    # Retrieve context using ColBERT search
    rag_results = RAG.search(query=question_text, k=11)
    # Retrieve context using BM25 search
    bm25_results = bm25_retriever.retrieve(question_text)
    context = combine_contexts(rag_results, bm25_results)

    abbreviations = find_appearing_abbreviations(q_data)
    # Generate the answer using the loaded model
    response = generate_answer(question_text, options, context, abbreviations, model, tokenizer)

    answer = parse_answer(response)

    # Extract the answer ID from the response
    match = re.search(r'Option (\d+)', answer)
    if match:
        try:
            answer_id = int(match.group(1))
            print(f"Answer ID: {answer_id}")
            responses.append([q_id_number, answer_id, "Phi-2"])
        except (KeyError, IndexError, ValueError) as e:
            responses.append([q_id_number, "Error", "Phi-2"])
            print(f"Error processing question {q_id}: {answer}")
    else:
        responses.append([q_id_number, "Error", "Phi-2"])
        print(f"Error processing question {q_id_number}: {answer}")

# Save responses to a CSV file
with open('output_results.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Question_ID", "Answer_ID", "Task"])
    csvwriter.writerows(responses)

print("Processing complete. Responses saved to 'output_results.csv'.")


In [ ]:
import csv
# Load the questions dataset from the JSON file
with open('data/366qs.txt', 'r') as rubric:
    qs_w_ans = json.load(rubric)

# Load the responses from the CSV file
responses = []
with open('output_results.csv', 'r') as answers:
    reader = csv.DictReader(answers)
    for row in reader:
        responses.append(row)

# Initialize score
correct_answers = 0
total_questions = len(responses)

# track question_ids for failed questions
failed_questions = []
# Compare the responses with the correct answers
for response in responses:
    question_id = response['Question_ID']
    answer_id = response['Answer_ID']
    task = response['Task']
    
    # Find the corresponding question in the JSON data
    question_key = f"question {question_id}"
    if question_key in qs_w_ans:
        correct_answer = qs_w_ans[question_key]['answer']
        # Extract the correct option number from the correct answer string
        correct_option_number = correct_answer.split()[1].replace(":", "")
        
        # Check if the given answer matches the correct answer
        if answer_id == correct_option_number:
            correct_answers += 1
        else:
            # append questionid and answerid to failed questions
            failed_questions.append((question_id, answer_id))

# Calculate the score
score = (correct_answers / total_questions) * 100

# Print the results
print(f"Total Questions: {total_questions}")
print(f"Correct Answers: {correct_answers}")
print(f"Score: {score:.2f}%")
# write failed questions to a file
with open('failed_questions.txt', 'w') as file:
    for question_id, answer_id in failed_questions:
        file.write(f"{question_id} {answer_id}\n")